In [1]:
import json
import pandas as pd
import random
import torch
from numerical_regressor import TokenNumericCollegeResultsDataset, CombinedDelayedRegressor, mappings

combined_data = {}
uncategorized_data = json.load(open('../../scraping/combined_collegeresults_data.json', 'r'))
standardized_output = json.load(open('../../categorization/standardized_output.json', 'r'))

colleges_list = open('../../categorization/all-colleges.txt').readlines()
colleges_list = [college[:college.index(' (')] for college in colleges_list]

college_data = pd.read_csv('../../categorization/college_acceptance.csv')

/Users/Yourui/Documents/nochances/nochances/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
major_data = pd.read_csv('../../categorization/major_data.csv')
major_data['combined'] = major_data[major_data.columns.drop(['Name', 'Total'])].values.tolist()
major_data = major_data[['Name', 'combined', 'Total']]
def to_frequencies(counts, total):
    return [float(count/total) if total else 0 for count in counts]
major_data['combined'] = major_data.apply(lambda x: to_frequencies(x['combined'], x['Total']), axis=1)

In [3]:
college_data = pd.merge(major_data, college_data, on="Name")
college_data

,Name,combined,Total,Applicants total,Admissions total,Enrolled total,Percent of freshmen submitting SAT scores,Percent of freshmen submitting ACT scores,SAT Critical Reading 25th percentile score,SAT Critical Reading 75th percentile score,...,Percent of freshmen receiving federal grant aid,Percent of freshmen receiving Pell grants,Percent of freshmen receiving other federal grant aid,Percent of freshmen receiving state/local grant aid,Percent of freshmen receiving institutional grant aid,Percent of freshmen receiving student loan aid,Percent of freshmen receiving federal student loans,Percent of freshmen receiving other loan aid,Endowment assets (year end) per FTE enrollment (GASB),Endowment assets (year end) per FTE enrollment (FASB)
0,Alabama A & M University,"[0.1782178217821782, 0.11287128712871287, 0.23...",505,6142.0,5521.0,1104.0,15.0,88.0,370.0,450.0,...,81.0,81.0,7.0,1.0,32.0,89.0,89.0,1.0,NaN,NaN
1,University of Alabama at Birmingham,"[0.10211565585331453, 0.051057827926657265, 0....",3545,5689.0,4934.0,1773.0,6.0,93.0,520.0,640.0,...,36.0,36.0,10.0,0.0,60.0,56.0,55.0,5.0,24136.0,NaN
2,University of Alabama in Huntsville,"[0.3168114460909555, 0.11088400613183444, 0.24...",1957,2054.0,1656.0,651.0,34.0,94.0,510.0,640.0,...,31.0,31.0,4.0,1.0,63.0,46.0,46.0,3.0,11502.0,NaN
3,Alabama State University,"[0.01282051282051282, 0.12307692307692308, 0.2...",390,10245.0,5251.0,1479.0,18.0,87.0,380.0,480.0,...,76.0,76.0,13.0,11.0,34.0,81.0,81.0,0.0,13202.0,NaN
4,The University of Alabama,"[0.13406237343053867, 0.015525853921965708, 0....",7407,30975.0,17515.0,6454.0,23.0,76.0,490.0,620.0,...,20.0,18.0,4.0,3.0,50.0,42.0,41.0,8.0,19469.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1261,Providence Christian College,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",21,122.0,65.0,20.0,NaN,NaN,NaN,NaN,...,50.0,50.0,14.0,0.0,100.0,64.0,64.0,14.0,NaN,350.0
1262,Polytechnic University of Puerto Rico-Orlando,"[0.64, 0.04, 0.32, 0.0, 0.0, 0.0, 0.0, 0.0, 0....",25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,100.0,100.0,0.0,50.0,0.0,50.0,50.0,0.0,NaN,NaN
1263,University of North Georgia,"[0.0031409501374165686, 0.06949352179034157, 0...",2547,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1264,Kennesaw State University,"[0.1752136752136752, 0.14213358657803102, 0.22...",6318,9471.0,5355.0,3194.0,67.0,45.0,500.0,590.0,...,36.0,36.0,2.0,68.0,4.0,51.0,51.0,3.0,1180.0,NaN


In [4]:
residences = [value['residence'] for value in standardized_output.values()]
all_numerical_data = [json.loads(line) for line in open('../../categorization/categorized.json', 'r').readlines()]
selective_enough = college_data.loc[college_data['Admissions total']/college_data['Applicants total'] < .50]['Name'].to_list()
selective_enough

['Oakwood University',
 'Spring Hill College',
 'Stillman College',
 'Tuskegee University',
 'Alaska Pacific University',
 'University of Arkansas at Pine Bluff',
 'California Institute of Technology',
 'California Lutheran University',
 'California Polytechnic State University-San Luis Obispo',
 'California State University-Fullerton',
 'California State University-Long Beach',
 'California State University-Northridge',
 'University of California-Berkeley',
 'University of California-Davis',
 'University of California-Irvine',
 'University of California-Los Angeles',
 'University of California-San Diego',
 'University of California-Santa Barbara',
 'Chapman University',
 'San Diego Christian College',
 'Claremont McKenna College',
 'Harvey Mudd College',
 'University of La Verne',
 'La Sierra University',
 'Menlo College',
 'Occidental College',
 'Hope International University',
 'Pepperdine University',
 'Pitzer College',
 'Pomona College',
 'San Diego State University',
 'University

In [5]:
fake_data_index = 0
for i, numerical_data in enumerate(all_numerical_data):

    if numerical_data == {'skip': True}:
        continue

    try:
        post_id = list(uncategorized_data.keys())[i]

        selected_data = [
            numerical_data['basic_info']['ethnicity'],
            numerical_data['basic_info']['gender'],
            numerical_data['basic_info']['income_bracket'],
            numerical_data['basic_info']['gpa'],
            numerical_data['basic_info']['ap_ib_courses'] ** (1/2),
            numerical_data['basic_info']['ap_ib_scores'],
            numerical_data['basic_info']['test_score'],
            numerical_data['basic_info']['location'],
            numerical_data['basic_info']['first_gen'],
        ] + list(numerical_data['ecs'].values()) + list(numerical_data['awards'].values())

        try:
            other_data = standardized_output[post_id]
            results = other_data['results']
            combined_data.update({post_id:{
                'major':other_data['major'],
                'residence':other_data['residence'],
                'ecs':other_data['extracurriculars'],
                'awards':other_data['awards'],
                'numeric':selected_data,
                'results':results
            }})
        except KeyError:
            continue
    
    # Change this to randomized data, except that the gpa is shit (~2.5 and selectivity is <50%)
    except IndexError:
        
        random_posts = []
        for _ in range(5):
            post = random.choice(all_numerical_data)
            while post == {'skip': True}:
                post = random.choice(all_numerical_data)
            random_posts.append(post)

        selected_data = [
            random.choice([0, 1]), ## Race
            random.choice([0, 1]), ## Gender
            random.choice([0, 1, 2, 3, 4]), ## Income Bracket
            random.choice([0, 1]), ## GPA
            random_posts[0]['basic_info']['ap_ib_courses'] ** (1/2),
            random_posts[1]['basic_info']['ap_ib_scores'],
            random_posts[2]['basic_info']['test_score'],
            random_posts[3]['basic_info']['location'],
            random_posts[4]['basic_info']['first_gen'],
        ] + list(numerical_data['ecs'].values()) + list(numerical_data['awards'].values())

        combined_data.update({i: {
            'major': mappings[random.randint(0,11)][0],
            'residence': random.choice(residences),
            'ecs': [],
            'awards': [],
            'numeric': selected_data,
            'results': [{
                'school_name': random.choice(selective_enough),
                'in_state': 0,
                'accepted': 0,
                'round': random.choice([1, 3]),
            } for _ in range(random.randint(1,5))]
        }})
        fake_data_index += 1

In [6]:
len(combined_data)

3081

In [7]:
# dataset = TokenNumericCollegeResultsDataset(combined_data, college_data)
# torch.save(dataset, 'shortened_numerical_fake_data.pt')
dataset = torch.load('shortened_numerical_fake_data.pt')

In [8]:
full_data_size = len(dataset)
train_size = int(full_data_size * 0.8)
print(f"Train Data Size: {train_size}")

gen = torch.Generator()
gen.manual_seed(0)

batch_size = 128
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, full_data_size - train_size], generator=gen)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=2)

Train Data Size: 17565


In [9]:
if torch.cuda.is_available():
    device = torch.device('cuda')
elif torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')
device

device(type='mps')

In [10]:
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, n_epochs, device):
    best_val_acc = 0
    for epoch in range(n_epochs):
        for param_group in optimizer.param_groups:
            print(f"Current learning rate: {param_group['lr']}")

        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0

        for i, batch in enumerate(train_loader):
            optimizer.zero_grad()

            for k, v in batch.items():
                batch[k] = v.to(device)

            outputs = model(batch['inputs'])
            loss = criterion(outputs, batch['target'])

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()

            train_loss += loss.item()
            train_total += batch['target'].size(0)
            train_correct += ((torch.sigmoid(outputs) > 0.5) == batch['target']).sum().item()

        train_loss /= len(train_loader)
        train_acc = train_correct / train_total

        # Validation
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for batch in val_loader:
                for k, v in batch.items():
                    batch[k] = v.to(device)

                outputs = model(batch['inputs'])
                loss = criterion(outputs, batch['target'])

                val_loss += loss.item()
                val_total += batch['target'].size(0)
                val_correct += ((torch.sigmoid(outputs) > 0.5) == batch['target']).sum().item()

            print(torch.sigmoid(outputs))

        val_loss /= len(val_loader)
        val_acc = val_correct / val_total

        scheduler.step(val_loss)

        print(f'Epoch {epoch+1}/{n_epochs}:')
        print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}')
        print(f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), 'reduced_v5_fake_data_numerical.pt')
            #v2 is 81% val acc

    return model

In [11]:
model = CombinedDelayedRegressor().to(device)

n_epochs = 40
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, n_epochs)
model = train_model(model, train_loader, test_loader, criterion, optimizer, scheduler, n_epochs, device)

Current learning rate: 0.0001
tensor([0.6465, 0.5544, 0.5914, 0.5384, 0.5403, 0.5722, 0.5562, 0.5582, 0.5630,
        0.5590, 0.5445, 0.5456, 0.6387, 0.5712, 0.5401, 0.5655, 0.5606, 0.5594,
        0.5506, 0.5442, 0.5712, 0.5376, 0.6107, 0.6090, 0.6114, 0.5412, 0.5452,
        0.5491, 0.5400, 0.6305, 0.5603, 0.5534, 0.5405, 0.6164, 0.5546, 0.5390,
        0.5473, 0.5641, 0.5595, 0.5717], device='mps:0')
Epoch 1/40:
Train Loss: 0.6802, Train Acc: 0.6049
Val Loss: 0.6645, Val Acc: 0.6061
Current learning rate: 9.993191928767829e-05


/Users/Yourui/Documents/nochances/nochances/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:156: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


tensor([0.8153, 0.6351, 0.5309, 0.5829, 0.5380, 0.6218, 0.5810, 0.7484, 0.7569,
        0.5316, 0.8866, 0.5311, 0.5222, 0.5686, 0.5540, 0.8055, 0.5854, 0.7917,
        0.5952, 0.5179, 0.5291, 0.6122, 0.8133, 0.5437, 0.5361, 0.5888, 0.5372,
        0.5277, 0.5165, 0.8093, 0.5280, 0.6501, 0.6919, 0.6097, 0.7437, 0.7003,
        0.7540, 0.5519, 0.5592, 0.6760], device='mps:0')
Epoch 2/40:
Train Loss: 0.6390, Train Acc: 0.6047
Val Loss: 0.6082, Val Acc: 0.6061
Current learning rate: 9.994296201012241e-05
tensor([0.9130, 0.4033, 0.4778, 0.8766, 0.6789, 0.9255, 0.9039, 0.5853, 0.4305,
        0.4966, 0.9134, 0.8119, 0.6078, 0.8985, 0.5129, 0.6430, 0.4790, 0.8977,
        0.5971, 0.6829, 0.4762, 0.6819, 0.6503, 0.4865, 0.5968, 0.7041, 0.4948,
        0.4709, 0.4656, 0.4372, 0.6150, 0.4887, 0.7035, 0.4168, 0.4535, 0.9118,
        0.4261, 0.8722, 0.8834, 0.9116], device='mps:0')
Epoch 3/40:
Train Loss: 0.6136, Train Acc: 0.6432
Val Loss: 0.5753, Val Acc: 0.6995
Current learning rate: 9.99489772